In [1]:
import numpy as np
import os
import re
import pandas as pd
import matplotlib.pylab as plt
import napari

from image_analysis.read_files import get_paths, change_paths
from image_analysis.fit import plot, roi, regionprops
from image_analysis.preprocess import mask

In [2]:
pixel_size = 8

In [3]:
folder_pretreated = r'example_pretreated_images'
df = pd.read_excel("example_images.xlsx")

In [4]:
df.head()

,Sample name,HCR,"Animal size, cm","Regeneration time, dpa","Exact animal size (snout to tail), cm"
0,hcr21_2_5cm_6dpa_noif,21,3,6,2.7
1,hcr21_2_5cm_6dpa_p_hist,21,3,6,2.7
2,hcr21_2_5cm_7dpa_noif,21,3,7,2.9


# Check rotation of SF to ey

In [5]:
i = 1
file_name = df['Sample name'].values[i]
    
file = os.path.join( folder_pretreated, file_name) + '.npy'

keys = np.load( file.split('.npy')[0] + '_keys.npy' )
image = dict(zip(
    keys,
    np.load( file )
))

pc = 50
if (f'shh{pc}_clip' in keys) and (f'fgf{pc}_clip' in keys):
    
    image = dict(zip(
        ['shh_mask', 'fgf_mask', 'tissue_mask_with_epi'],
        [ (image[f'shh{pc}_clip'] > 0).astype(int), (image[f'fgf{pc}_clip']>0).astype(int), image['tissue_mask_with_epi'] ]
    ))
    
    image['shh_mask'] = mask.largest_component( image['shh_mask'] )
    image['fgf_mask'] = mask.largest_component( image['fgf_mask'] )

    regionprops_3d = regionprops.regionprops_3d(image, 1, 1, 1, pixel_size)
    image = regionprops_3d.rotate_sf_to_ey()
    regionprops_3d.center_mass(['shh_mask', 'fgf_mask'])
    regionprops_3d.tensor_inertia('tissue_mask_with_epi')

/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:24: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:27: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:30: RuntimeWarning: Mean of empty slice
  img_arr_z = np.nanmean(np.nanmean(img_arr, axis=1), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:24: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:27: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/D

Re-running the rotation function due to insufficient alignment
Re-running the rotation function due to insufficient alignment
Angle between ey and SF: 2


In [6]:
scale_u = (pixel_size, pixel_size, pixel_size)
viewer = napari.Viewer()
viewer.add_image(image['tissue_mask_with_epi'], name='tissue_mask_with_epi', scale=scale_u)
viewer.add_image(image['shh_mask'], name='shh_mask', gamma=0.2, opacity=0.5, colormap="red", scale=scale_u)
viewer.add_image(image['fgf_mask'], name='fgf_mask', gamma=0.2, opacity=0.5, colormap="blue", scale=scale_u)

viewer.axes.visible = True
viewer.scale_bar.visible = True
viewer.scale_bar.unit = "um"

viewer.add_points(regionprops_3d.parm['cm_shh_mask'], name='cm_shh', face_color='red')
viewer.add_points(regionprops_3d.parm['cm_fgf_mask'], name='cm_fgf', face_color='blue')

vector_line = np.array([ regionprops_3d.parm['cm_shh_mask'], regionprops_3d.parm['cm_fgf_mask'] ])
viewer.add_shapes(vector_line, shape_type='line', name='SF', edge_color='red')

ey = np.array([0,-1,0])
vector_line = np.array([ regionprops_3d.parm['cm_shh_mask'], regionprops_3d.parm['cm_shh_mask'] - 100*ey ])
viewer.add_shapes(vector_line, shape_type='line', name='ey', edge_color='green')

<Shapes layer 'ey' at 0x7e35e16d09a0>

# Loop trough all samples

In [7]:
for pc in [50]:

    df[f'system_size_L_along_SF_pc{pc}_um']  = pd.NA
    df[f'length_L_SF_pc{pc}_um']  = pd.NA
    
    df[f'x0_pc{pc}_um_shh']  = pd.NA
    df[f'x0_pc{pc}_um_fgf'] = pd.NA

In [8]:
for i, file_name in enumerate( df['Sample name'].values):

    print(i, file_name)
    
    file = os.path.join( folder_pretreated, file_name) + '.npy'
    
    keys = np.load( file.split('.npy')[0] + '_keys.npy' )
    image = dict(zip(
        keys,
        np.load( file )
    ))

    for pc in [50]:

        if (f'shh{pc}_clip' in keys) and (f'fgf{pc}_clip' in keys):

            print(pc)
            
            image_to_rotate = dict(zip(
                ['shh_mask', 'fgf_mask', 'tissue_mask_with_epi'],
                [ (image[f'shh{pc}_clip'] > 0).astype(int), (image[f'fgf{pc}_clip']>0).astype(int), image['tissue_mask_with_epi'] ]
            ))
            
            image_to_rotate['shh_mask'] = mask.largest_component( image_to_rotate['shh_mask'] )
            image_to_rotate['fgf_mask'] = mask.few_largest_components( image_to_rotate['fgf_mask'], 2 )
             
            regionprops_3d = regionprops.regionprops_3d(image_to_rotate, 1, 1, 1, pixel_size)
            image_to_rotate = regionprops_3d.rotate_sf_to_ey()

            S = regionprops.center_mass( image_to_rotate['shh_mask'] )
            F = regionprops.center_mass( image_to_rotate['fgf_mask'] )

            L = np.sum( 
                mask.largest_component( image_to_rotate['tissue_mask_with_epi'][S[0], :, S[2]]  )
            ) * pixel_size
            L_SF  = np.abs( S[1] - F[1] ) * pixel_size
            
            ymin = np.nonzero( mask.largest_component(image_to_rotate['tissue_mask_with_epi'][S[0], :, S[2]]) )[0][0]
            ymax = np.nonzero( mask.largest_component(image_to_rotate['tissue_mask_with_epi'][S[0], :, S[2]]) )[0][-1]            
            
            x0_S = min( (S[1] - ymin) * pixel_size, (ymax - S[1]) * pixel_size )
            x0_F = min( (F[1] - ymin) * pixel_size, (ymax - F[1]) * pixel_size )

            df.loc[df['Sample name'] == file_name, f'system_size_L_along_SF_pc{pc}_um']  = L
            df.loc[df['Sample name'] == file_name, f'length_L_SF_pc{pc}_um']             = L_SF
            df.loc[df['Sample name'] == file_name, f'x0_pc{pc}_um_shh']                  = x0_S
            df.loc[df['Sample name'] == file_name, f'x0_pc{pc}_um_fgf']                  = x0_F
            
df.to_excel("results/system_size_L_and_gaps_x0_from_3d.xlsx", index=False)

0 hcr21_2_5cm_6dpa_noif
50
Angle between ey and SF: 0
1 hcr21_2_5cm_6dpa_p_hist
50
Re-running the rotation function due to insufficient alignment
Re-running the rotation function due to insufficient alignment
Angle between ey and SF: 2
2 hcr21_2_5cm_7dpa_noif
50


/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:24: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:27: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:30: RuntimeWarning: Mean of empty slice
  img_arr_z = np.nanmean(np.nanmean(img_arr, axis=1), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:24: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:27: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/D

Angle between ey and SF: 1
